## Escalado en Python

De https://github.com/scidatmath2020/ML_Py_25/tree/main/data descarga la tabla ``datos_para_escalado.csv``. Con ella, veremos cómo se hace el escalado de columnas numéricas en Python.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer

datos = pd.read_csv("datos_para_escalar_ALUMNOS.csv")
datos.columns

Index(['temperatura_corp', 'frecuencia_cardiaca', 'peso_kg',
       'humedad_relativa_pct', 'ventas_mensuales', 'gastos_publicidad',
       'tipo_cliente', 'canal_venta', 'ciudad'],
      dtype='object')

In [2]:
datos

,temperatura_corp,frecuencia_cardiaca,peso_kg,humedad_relativa_pct,ventas_mensuales,gastos_publicidad,tipo_cliente,canal_venta,ciudad
0,38.4,161,81.8,86,30912.0,9709.0,Nuevo,Distribuidor,Mérida
1,36.5,124,117.0,46,78937.0,14900.0,Nuevo,Distribuidor,Mérida
2,36.2,119,84.4,15,61576.0,14538.0,Nuevo,Distribuidor,Monterrey
3,37.7,130,90.9,59,60600.0,4787.0,Recurrente,Distribuidor,Mérida
4,38.5,89,56.4,40,59988.0,13985.0,Nuevo,Distribuidor,Guadalajara
5,37.1,113,105.8,25,39386.0,12349.0,Recurrente,Online,Mérida
6,39.7,65,90.2,45,60063.0,12302.0,Nuevo,Tienda,CDMX
7,38.4,77,86.2,49,500000.0,13121.0,Recurrente,Tienda,CDMX
8,37.4,132,72.0,0,32557.0,14942.0,VIP,Online,Querétaro
9,37.0,73,69.3,35,60334.0,120000.0,VIP,Online,Monterrey


In [3]:
num_cols = datos.select_dtypes(include=[np.number]).columns

cols_con_outliers = []
for c in num_cols:
    s = pd.to_numeric(datos[c], errors="coerce").dropna()
    if s.empty:
        continue
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    if iqr == 0:
        continue
    lim_inf, lim_sup = q1 - 1.5*iqr, q3 + 1.5*iqr
    n_out = int(((s < lim_inf) | (s > lim_sup)).sum())
    if n_out > 0:
        cols_con_outliers.append(c)

cols_con_outliers

['ventas_mensuales', 'gastos_publicidad']

In [4]:
# Definir columnas por tipo de escalado
cols_std = ["temperatura_corp", "frecuencia_cardiaca", "peso_kg"]
cols_minmax = ["humedad_relativa_pct"]
cols_robust = ["ventas_mensuales", "gastos_publicidad"]

In [ ]:
# Construir el transformador por columnas

'''
Usamos ColumnTransformer para aplicar **distintas transformaciones a distintos subconjuntos 
de columnas en una sola pasada. Cada tupla dentro de transformers=[...] 
tiene la forma (nombre_bloque, transformador, columnas_objetivo).

- ("std", StandardScaler(), cols_std)  
  Aplica estandarización a las columnas listadas en cols_std
- ("minmax", MinMaxScaler(), cols_minmax)  
  Aplica normalización Min–Max a las columnas de cols_minmax
- ("robust", RobustScaler(quantile_range=(25, 75)), cols_robust)  
  Aplica escalado robusto a las columnas de cols_robust

Parámetros adicionales del `ColumnTransformer`:

- remainder="drop"  
  Indica que las columnas no listadas en los bloques anteriores se descartan de la salida. 
  Alternativa: "passthrough" para dejarlas pasar sin transformar)

- verbose_feature_names_out=False  
  Mantiene nombres de salida “limpios” (p. ej., `edad`) en lugar de prefijarlos con el nombre 
  del bloque (p. ej., `std__edad`).
'''

preprocesador = ColumnTransformer(
    transformers=[
        ("std", StandardScaler(), cols_std),
        ("minmax", MinMaxScaler(), cols_minmax),
        ("robust", RobustScaler(quantile_range=(25, 75)), cols_robust),
    ],
    remainder="drop",  
    verbose_feature_names_out=False
)

# Ajustar (fit) sobre TODO el dataset
preprocesador.fit(datos)

# Transformar con los parámetros aprendidos
X_escalado = preprocesador.transform(datos)

# Reconstruir DataFrame con los mismos nombres de columnas

columnas_escaladas = preprocesador.get_feature_names_out()
datos_escalado = pd.DataFrame(X_escalado, columns=columnas_escaladas, index=datos.index)
datos_escalado

In [8]:
# --- Reordenar para que siga el orden original del CSV ---
orden_original = [c for c in datos.columns if c in datos_escalado.columns]
datos_escalado = datos_escalado[orden_original]
datos_escalado

,temperatura_corp,frecuencia_cardiaca,peso_kg,humedad_relativa_pct,ventas_mensuales,gastos_publicidad
0,0.896248,1.244719,-0.201124,0.924731,-0.977494,-0.134884
1,-0.968210,0.234750,1.770897,0.494624,0.651493,0.749744
2,-1.262598,0.098267,-0.055463,0.161290,0.062616,0.688054
3,0.209343,0.398528,0.308688,0.634409,0.029510,-0.973671
4,0.994378,-0.720627,-1.624116,0.430108,0.008751,0.593814
5,-0.379434,-0.065512,1.143436,0.268817,-0.690060,0.315014
6,2.171930,-1.375742,0.269472,0.483871,0.011295,0.307004
7,0.896248,-1.048184,0.045379,0.526882,14.933772,0.446575
8,-0.085045,0.453121,-0.750152,0.000000,-0.921697,0.756902
9,-0.477563,-1.157370,-0.901415,0.376344,0.020487,18.660446


In [9]:
datos_escalado.to_csv("datos_reescalados_alumnos.csv")